In [14]:
import torch
import numpy as np

In [162]:
torch.cuda.is_available()


False

In [57]:
def create_gaussian_particles(mean, std, N):
    particles = torch.empty((N, 3))
    particles[:, 0] = mean[0] + (torch.randn(N) * std[0])
    particles[:, 1] = mean[1] + (torch.randn(N) * std[1])
    particles[:, 2] = mean[2] + (torch.randn(N) * std[2])
    particles[:, 2] %= 2 * np.pi
    return particles

In [52]:
torch.distributions.multivariate_normal.MultivariateNormal(particles[0, 0:2], covariance_matrix=torch.diag(torch.FloatTensor([1,1]))).log_prob(torch.FloatTensor([1,1]))

tensor(-2.8379)

In [43]:
torch.FloatTensor([1,1])

tensor([1., 1.])

In [54]:
def update(particles, weights, z, R, landmarks):
    for i, landmark in enumerate(landmarks):
        distance = torch.norm(particles[:, 0:2] - landmark, dim=1)
        weigths *= torch.FloatTensor(10).pow(torch.distributions.multivariate_normal.MultivariateNormal(distance, covariance_matrix=R).log_prob(z[i]))

    weights += torch.Tensor([1.e-300])     # avoid round-off to zero
    weights /= sum(weights) # normalize

In [187]:
def predict(particles, u, std, dt=torch.Tensor([1.])):
    """ move according to control input u (heading change, velocity)
    with noise Q (std heading change, std velocity)`"""

    N = len(particles)
    # update heading
    particles[:, 2] += u[0] + (torch.randn(N) * std[0])
    particles[:, 2] %= torch.Tensor([2 * np.pi])

    # move in the (noisy) commanded direction
    dist = (u[1] * dt) + (torch.randn(N) * std[1])
    particles[:, 0] += torch.cos(particles[:, 2]) * dist
    particles[:, 1] += torch.sin(particles[:, 2]) * dist
    return particles

In [144]:
def estimate(particles, weights):
    """returns mean and variance of the weighted particles"""

    pos = particles[:, 0:2]
    mean = torch.sum(torch.mul(pos,weights),dim=0)/torch.sum(weights)
    var = torch.sum(torch.mul((pos - mean).pow(2),weights),dim=0)/torch.sum(weights)
    return mean, var

In [229]:
def neff(weights):
    return 1. / torch.sum(torch.mul(weights,weights))

In [230]:
N = 30000
particles = create_gaussian_particles([4,3,1],[1,2,1], N)
weights = torch.ones(N,1)* 0.25
estimate(particles, weights)
neff(weights)

tensor(0.0005)